In [ ]:
import os
import torch
import numpy as np
import faiss
from collections import defaultdict
from neo4j import GraphDatabase

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

print("🔄 Loading data safely...")
saved = torch.load("full_graph.pt", weights_only=False)
node_meta = saved['node_meta']
NODE_TYPE_MAP = saved['NODE_TYPE_MAP']

sw_embeddings = np.load("sw_embeddings.npy")

sw_mask = (saved['data'].node_type == NODE_TYPE_MAP['SceneWindow'])
sw_indices = sw_mask.nonzero(as_tuple=True)[0]
del saved

print("📊 Computing movie centroids...")
movie_to_sw_indices = defaultdict(list)
for idx in range(len(sw_indices)):
    meta = node_meta[sw_indices[idx].item()]
    movie_to_sw_indices[meta['movie']].append(idx)

movie_centroids = {}
for movie_title, indices in movie_to_sw_indices.items():
    vectors = sw_embeddings[indices].astype(np.float32)
    movie_centroids[movie_title] = np.mean(vectors, axis=0)

print("🏗️  Building FAISS index...")
movie_titles = list(movie_centroids.keys())
movie_matrix = np.array([movie_centroids[t] for t in movie_titles], dtype=np.float32)
faiss.normalize_L2(movie_matrix)

movie_index = faiss.IndexFlatIP(256)
movie_index.add(movie_matrix)

print("🚀 Running recommendation pipeline...")

URI = "neo4j+ssc://2bdf163a.databases.neo4j.io:7687"
AUTH = ("neo4j", "dVRqLgBpDBT3tP37uYphK_zFZRjBHRizDVvRc4LCJRg")

def get_structural_logic(target_movie, recommended_movie):
    try:
        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            with driver.session() as session:
                query = """
                MATCH (m1:Movie {title: $m1})-[:HAS_WINDOW]->(sw1:SceneWindow)
                MATCH (m2:Movie {title: $m2})-[:HAS_WINDOW]->(sw2:SceneWindow)
                MATCH (sw1)-[:HAS_TROPE|IN_PHASE|HAS_EMOTION|HAS_ARCHETYPE]->(common)
                MATCH (sw2)-[:HAS_TROPE|IN_PHASE|HAS_EMOTION|HAS_ARCHETYPE]->(common)
                RETURN labels(common)[0] as type, common.name as name, count(*) as weight
                ORDER BY weight DESC LIMIT 5
                """
                return session.run(query, m1=target_movie, m2=recommended_movie).data()
    except Exception as e:
        return []

def run_safe_pipeline(target_movie):
    # 1. Search
    if target_movie not in movie_centroids:
        return "Movie not found"
        
    query_vec = movie_centroids[target_movie].reshape(1, -1).astype(np.float32)
    faiss.normalize_L2(query_vec)
    D, I = movie_index.search(query_vec, k=4)
    
    # 2. Context Builder
    report_lines = []
    for dist, idx in zip(D[0], I[0]):
        sim_movie = movie_titles[idx]
        if sim_movie == target_movie: continue
        
        commons = get_structural_logic(target_movie, sim_movie)
        traits = [f"{c['name']}({c['type']})" for c in commons]
        
        report_lines.append(
            f"▶ 추천 영화: {sim_movie} (유사도: {dist:.4f})\n"
            f"  - 핵심 연결고리: {', '.join(traits)}"
        )
        
    return "\n".join(report_lines)

# Run
final_result = run_safe_pipeline("Aftersun 2022")
print("\n=== ✨ Final Recommendation Result ✨ ===")
print(final_result)

🔄 Loading data safely...
📊 Computing movie centroids...
🏗️  Building FAISS index...
🚀 Running recommendation pipeline...

=== ✨ Final Recommendation Result ✨ ===
▶ 추천 영화: Passing 2021 (유사도: 0.9901)
  - 핵심 연결고리: Rising(NarrativePhase), Sadness(Emotion), Discovery(Trope), Quest(Trope), Fear(Emotion)
▶ 추천 영화: The Whale 2022 (유사도: 0.9873)
  - 핵심 연결고리: Sadness(Emotion), Rising(NarrativePhase), Quest(Trope), Discovery(Trope), Joy(Emotion)
▶ 추천 영화: The Father 2020 (유사도: 0.9863)
  - 핵심 연결고리: Rising(NarrativePhase), Discovery(Trope), Sadness(Emotion), Quest(Trope), Fear(Emotion)


In [ ]:
import openai
import os

# 1. API 키 설정
api_key = os.getenv("OPENAI_API_KEY") 
if not api_key:
    api_key = "YOUT_OPENAI_KEY"
client = openai.OpenAI(api_key=api_key)

def get_final_recommendation_report(target_movie_title):
    # 1단계: 이전 단계에서 만든 파이프라인 함수(run_safe_pipeline) 호출
    print(f"🔍 Searching for structural logic for: {target_movie_title}...")
    structured_prompt = run_safe_pipeline(target_movie_title)
    
    if not structured_prompt or "Movie not found" in structured_prompt:
        return f"❌ '{target_movie_title}'에 대한 분석 데이터를 찾을 수 없어 리포트를 생성할 수 없습니다."

    try:
        # 2단계: ChatGPT API 호출 (gpt-4o 권장)
        print("🤖 Generating narrative analysis with GPT-4o...")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system", 
                    "content": "당신은 지식 그래프와 GNN 분석 데이터를 해석하여 영화를 추천하는 서사 분석 전문가입니다. 제공된 구조적 유사성 데이터(SceneWindow, Trope, Emotion 등)를 바탕으로, 왜 이 영화들이 추천되었는지 독자가 이해하기 쉽게 서사적 함의를 풀어서 설명하는 통찰력 있는 리포트를 작성하세요."
                },
                {
                    "role": "user", 
                    "content": f"다음은 '{target_movie_title}'와 유사한 영화들의 구조적 분석 데이터입니다:\n\n{structured_prompt}\n\n이 데이터를 바탕으로 추천 리포트를 작성해줘."
                }
            ],
            temperature=0.7
        )
        
        # 3단계: 최종 리포트 출력
        return response.choices[0].message.content

    except Exception as e:
        return f"❌ API 호출 중 오류 발생: {e}"

# 🔥 실행: Aftersun 2022를 기반으로 한 서사 분석 추천 결과 보기
final_report = get_final_recommendation_report("Aftersun 2022")

print("\n==================================================")
print(f"🎬 Aftersun 2022 기반 서사 구조 분석 추천 리포트")
print("==================================================\n")
print(final_report)

🔍 Searching for structural logic for: Aftersun 2022...
🤖 Generating narrative analysis with GPT-4o...

🎬 Aftersun 2022 기반 서사 구조 분석 추천 리포트

추천 영화들이 'Aftersun 2022'와 공유하는 구조적 유사성을 중심으로 서사적 함의를 풀어보겠습니다. 이 영화들은 모두 감정적으로 깊이 있는 서사를 통해 인간 경험의 복잡성을 탐구합니다.

### Passing 2021

- **Rising(NarrativePhase)**: 이 영화는 서서히 상승하는 서사 구조를 통해 긴장감을 쌓아가며, 인물들이 직면하는 사회적, 개인적 도전들을 조명합니다. 이는 'Aftersun'에서의 인물들이 관계를 통해 성장하는 과정과 유사합니다.
  
- **Sadness(Emotion) & Fear(Emotion)**: 두 영화 모두 감정적으로 깊은 슬픔과 불안감을 탐구합니다. 'Passing'은 인종적 정체성의 혼란과 그로 인한 내적 갈등을, 'Aftersun'은 부모와 자녀 간의 관계에서 오는 감정적 복잡성을 다룹니다.

- **Discovery & Quest(Trope)**: 인물들이 자신의 정체성을 찾고자 하는 여정이 중심에 있습니다. 'Passing'에서는 사회적 규범과 기대에 대한 도전이 주제를 이루며, 'Aftersun'은 개인적 기억과 관계의 이해를 탐구합니다.

### The Whale 2022

- **Sadness(Emotion) & Joy(Emotion)**: 이 영화는 슬픔과 기쁨이 교차하는 복잡한 감정을 통해 인물의 내면을 깊이 파고듭니다. 'Aftersun'과 유사하게, 감정의 파노라마를 통해 인물의 변화를 섬세하게 그립니다.

- **Rising(NarrativePhase) & Quest(Trope)**: 인물의 내적 여정과 성장 과정이 서사의 중심을 이룹니다. 'The Whale'은 주인공의 자기 수용과 사랑을 찾는 과정을 그리며, 이는 'Aftersu